<a href="https://colab.research.google.com/github/Medhavi-16/Video-Stegnography/blob/main/video_stego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stegano
!pip install pyaes
!pip install pbkdf2

In [ ]:
import pyaes, pbkdf2, binascii, os, secrets
password = "s3cr3t*c0d3"
passwordSalt = os.urandom(16)
key = pbkdf2.PBKDF2(password, passwordSalt).read(32)
print('AES encryption key:', binascii.hexlify(key))
iv = secrets.randbits(256)

AES encryption key: b'f986a760e9bcd91c89b45ff84ae9d143ea12fb25ad8dec6a847fe7abc2e8863c'


In [ ]:
from stegano import lsb
from os.path import isfile,join

import time    
import cv2
import numpy as np
import math
import os
import shutil
from subprocess import call,STDOUT
import zlib

def split_string(s_str,count=10):
    per_c=math.ceil(len(s_str)/count)
    c_cout=0
    out_str=''
    split_list=[]
    for s in s_str:
        out_str+=s
        c_cout+=1
        if c_cout == per_c:
            split_list.append(out_str)
            out_str=''
            c_cout=0
    if c_cout!=0:
        split_list.append(out_str)
    return split_list

def frame_extraction(video):
    if not os.path.exists("./tmp"):
        os.makedirs("tmp")
    temp_folder="./tmp"
    print("[INFO] tmp directory is created")

    vidcap = cv2.VideoCapture(video)
    count = 0

    while True:
        success, image = vidcap.read()
        if not success:
            break
        cv2.imwrite(os.path.join(temp_folder, "{:d}.png".format(count)), image)
        count += 1


def compress(s_str):
    compressed_data=zlib.compress(s_str.encode('utf-8'),2)
    compressed_data_text=binascii.hexlify(compressed_data).decode('utf-8')
    return compressed_data_text

def decompress(s_str):
    decompressed_data=zlib.decompress(binascii.unhexlify(s_str.encode('utf-8'))).decode('utf-8')
    return decompressed_data

def encrypt(s_str):
    plaintext = s_str
    aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
    ciphertext = aes.encrypt(plaintext)
    ans=binascii.hexlify(ciphertext)
    print('Encrypted:', ans)
    print(type(ans))
    ans1=ans.decode('utf-8')
    print(ans1)
    print(type(ans1))
    return ans1


def decrypt(s_str):
    aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
    decrypted = aes.decrypt(binascii.unhexlify(s_str.encode('utf-8')))
    print('Decrypted:', decrypted)
    return decrypted.decode('utf-8')
    
def encode_string(input_string,root="./tmp/"):
    ans=compress(input_string)
    ans=encrypt(ans)
    split_string_list=split_string(ans)
    for i in range(0,len(split_string_list)):
        f_name="{}{}.png".format(root,i)
        secret_enc=lsb.hide(f_name,split_string_list[i])
        secret_enc.save(f_name)
        print("[INFO] frame {} holds {}".format(f_name,split_string_list[i]))
def decode_string(video):
    frame_extraction(video)
    secret=[]
    root="./tmp/"
    for i in range(len(os.listdir(root))):
        f_name="{}{}.png".format(root,i)
        secret_dec=lsb.reveal(f_name)
        if secret_dec == None:
            break
        secret.append(secret_dec)
    
    s_str = ''.join([i for i in secret])
    print(decompress(decrypt(s_str)))
    clean_tmp()

def clean_tmp(path="./tmp"):
    if os.path.exists(path):
        shutil.rmtree(path)
        print("[INFO] tmp files are cleaned up")

def main():
    input_string = input("Enter the input string :")
    f_name=input("enter the name of video")
    frame_extraction(f_name)
    call(["ffmpeg", "-i",f_name, "-q:a", "0", "-map", "a", "tmp/audio.mp3", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)
    
    encode_string(input_string)
    call(["ffmpeg", "-i", "tmp/%d.png" , "-vcodec", "png", "tmp/video.mov", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)
    
    call(["ffmpeg", "-i", "tmp/video.mov", "-i", "tmp/audio.mp3", "-codec", "copy", "video.mov", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)
    clean_tmp()
if __name__ == "__main__":
    while True:
        print("1.Hide a message in video 2.Reveal the secret from video")
        print("any other value to exit")
        choice = input()
        if choice == '1':
            main()
        elif choice == '2':
            decode_string(input("enter the name of video with extension"))
        else:
            break